In [3]:
import csv
import os
import pickle

import shutil
import tqdm

import pandas as pd

In [6]:
data_path = '/data/zak/rosbag/labeled'

In [ ]:
def get_labels(joint_filenames):
    labels = []
    for joint_path in joint_filenames:
        with open(joint_path, 'rb') as joint_file:
            vel = pickle.load(joint_file)['velocity']
            front_avg_vel = (vel[2] + vel[3]) / 2
            if front_avg_vel > 1:
                label = 'no_obstacle'
            elif -0.1 < front_avg_vel < 0.1:
                label = 'obstacle'
            else:
                label = 'unknown'
        labels.append(label)
    return labels

In [ ]:
def create_csv(img_filenames, joint_filenames, dataset_path):
    labels = get_labels(joint_filenames)
    path_labels = list(zip(img_filenames, labels))
    print(f'Writing labels.csv to {dataset_path}')
    with open(os.path.join(dataset_path, 'labels.csv'), 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['img_path', 'label'])
        writer.writerows(path_labels)

In [ ]:
for dataset in os.listdir(data_path):
    dataset_path = os.path.join(data_path, dataset)    

    # Sort all img and joints contents in ascending order and put them on a csv file with the obtained label
    img_filenames = sorted(os.listdir(os.path.join(dataset_path, 'img')))
    img_filenames = [os.path.join(dataset_path, 'img', img) for img in img_filenames]
    joint_filenames = sorted(os.listdir(os.path.join(dataset_path, 'joints')))
    joint_filenames = [os.path.join(dataset_path, 'joints', joint) for joint in joint_filenames]
    create_csv(img_filenames, joint_filenames, dataset_path)
    

# Split datasets based on the labels generated to test the quality

In [7]:
extracted_data_path = '/data/zak/rosbag/extracted'
for dataset in os.listdir(data_path):
    if dataset not in ['heracleia', 'uc', 'mocap']:
        continue
    csv_path = os.path.join(data_path, dataset, 'labels.csv')
    new_dataset_path = os.path.join(extracted_data_path, dataset)
    with open(csv_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row in enumerate(csv_reader):
            if i == 0:
                continue
            # print(row)
            img_path = row[0]
            img_label = row[1]
            img_name = img_path.split('/')[-1]
            new_img_path = os.path.join(new_dataset_path, img_label, img_name)
            os.makedirs(os.path.dirname(new_img_path), exist_ok=True)
            shutil.copy(img_path, new_img_path)

# Read from a csv file and check distribution

In [24]:
import pandas as pd

In [11]:
csv_path = '/data/zak/rosbag/labeled/heracleia/labels.csv'

In [12]:
df = pd.read_csv(csv_path, delimiter=',')

In [13]:
df.head()

,img_path,label
0,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
1,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
2,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
3,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
4,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle


In [19]:
df['label'].value_counts()

no_obstacle    5815
unknown        1521
obstacle       1514
Name: label, dtype: int64

## Combine multiple csv files

In [20]:
mocap_csv_path = '/data/zak/rosbag/labeled/mocap/labels.csv'

In [21]:
mocap_df = pd.read_csv(mocap_csv_path, delimiter=',')

In [22]:
mocap_df.head()

,img_path,label
0,/data/zak/rosbag/labeled/mocap/img/16615524345...,obstacle
1,/data/zak/rosbag/labeled/mocap/img/16615524345...,obstacle
2,/data/zak/rosbag/labeled/mocap/img/16615524346...,obstacle
3,/data/zak/rosbag/labeled/mocap/img/16615524347...,obstacle
4,/data/zak/rosbag/labeled/mocap/img/16615524348...,obstacle


In [23]:
mocap_df['label'].value_counts()

no_obstacle    5094
obstacle       1991
unknown        1406
Name: label, dtype: int64

In [26]:
comb_df = pd.concat([df, mocap_df])
comb_df.head()

,img_path,label
0,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
1,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
2,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
3,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle
4,/data/zak/rosbag/labeled/heracleia/img/1661551...,obstacle


In [29]:
comb_df['label'].value_counts()

no_obstacle    10909
obstacle        3505
unknown         2927
Name: label, dtype: int64

## Generate Labels with Joints and Directions

In [25]:
data_path = '/data/zak/robot/labels'

In [47]:
# for dataset in os.listdir(data_path):
for dataset in ['heracleia']:
    if '.ipynb_checkpoints' == dataset:
        continue
    info_path = os.path.join(data_path, dataset, 'info.csv')
    info_df = pd.read_csv(info_path)
    labeled_data = info_df.loc[info_df['jointLabeled'] == True]
    break

In [48]:
labeled_data.head()

,Unnamed: 0,dataset,date,jointLabeled,img,joints,front_laser,label
6252,6252,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6253,6253,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6254,6254,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6255,6255,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6256,6256,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN


In [49]:
# Drop 'Unnamed: 0' Column and 'label' column
labeled_data = labeled_data.loc[:, ~labeled_data.columns.str.contains('^Unnamed')]
labeled_data.head()

,dataset,date,jointLabeled,img,joints,front_laser,label
6252,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6253,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6254,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6255,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN
6256,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,NaN


In [50]:
def get_direction(joint_path):
    with open(joint_path, 'rb') as f:
        values = pickle.load(f)
    velocities = values['velocity']
    BL, BR, FL, FR = velocities
    # Check stop condition
    if all([-0.5 < v < 0.5 for v in velocities]):
        return 'STOP'
    elif all([v > 2 for v in velocities]):
        return 'FORWARD'
    elif all([FL > FR, BL > BR]):
        return 'RIGHT'
    elif all([FL < FR, BL < BR]):
        return 'LEFT'
    else:
        return 'UNKNOWN'

In [51]:
labeled_data['label'] = labeled_data.joints.apply(lambda joint_path: get_direction(joint_path))

In [52]:
labeled_data.head()

,dataset,date,jointLabeled,img,joints,front_laser,label
6252,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,STOP
6253,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,STOP
6254,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,STOP
6255,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,STOP
6256,heracleia,8_26,True,/data/zak/robot/extracted/heracleia/8_26/img/1...,/data/zak/robot/extracted/heracleia/8_26/joint...,/data/zak/robot/extracted/heracleia/8_26/front...,STOP


In [53]:
labeled_data.label.value_counts()

FORWARD    3821
RIGHT      1553
LEFT       1451
STOP       1269
UNKNOWN     756
Name: label, dtype: int64